In [1]:
print("Trial graph for all the agents")

Trial graph for all the agents


IMPORTING ALL THE REQUIRED PACKAGES

In [2]:
!pip install -qU google-genai langchain openai python-dotenv beautifulsoup4 matplotlib networkx

In [3]:
import research_agent  , question_generator_agent , coding_agent , research_2

d:\LangChain\DSA Helpr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
C:\Users\DELL\AppData\Local\Temp\ipykernel_5616\2486430593.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.

In [4]:
from typing import TypedDict, List
import matplotlib.pyplot as plt

CREATING THE GRAPH STATE ---> this is a shared data source in the entire graph

In [5]:
class State(TypedDict):
    topic:str # this is the main concept/algorithm/idea to research and frame questions on 
    research_json: dict # this will hold the json output from the research agent
    question: dict # this is formatted data of the question that is generated by the question generator agent
    solution: dict # this field is the answer to the problem , need to be filled optionally
    human_feedback: str # this field is ment to add human feedback , mostly regarding to the question 

Creating a simple researcher node

In [6]:
from research_2 import JsonOutput , ResearchAgent


def research_node(state : State):
    """ this is a node for the researcher agent , to perform web search and curate
    information on the requested topic"""

    
    print("********* Researcher node is running**********")
    topic = state['topic']

    # create the research object and call the functions
    research_agent = ResearchAgent()
    output = research_agent.search(topic)
    print(f"the final output got from the research agent  is -->\n\n {output}" )

    # the final step is to add the data or any changes to the graph state
    return {"research_json": output}

Creating the question creator node

In [7]:
def question_crafter_node(state : State):
    """  This node is to be used for crafting the question from the research json object
    Here the key insights from the research will be transformed into a question after analyzing it and taking into consideration all the key technical details.
    """

    print("*********** Question Crafter node is running ****************")

    #1. take the research json object from the graph state
    research_json = state['research_json'] # this is the knowledge packet for creating the question

    # serialize this data
    import json
    from question_generator import QuestionGenerator
    question_creator = QuestionGenerator()
    research_string_data = json.dumps(research_json)
    generated_question = question_creator.generate_question(research_string_data)
    print(f"\n The question generated by the agent is : \n\n {generated_question}")
    #update the graph state with this question
    return {"question": generated_question.model_dump()}

Creating the coder agent 

In [8]:
def question_solver(state:State):
    """ This node is to be used for solving the question that is generated by the question generator agent
    This will involve analyzing the question and generating a solution based on the constraints given and generating the most optimal solution.
    """

    print("*********** Question Solver node is running ****************")

    #1. take the research json object from the graph state
    question = state['question'] 
    # this data needs to be converted into the pydantic model 
    from question_generator_agent import CodeQuestion # this is the data model
    import coding_agent
    formatted_question = CodeQuestion(**question)
    #now give this to the llm to solve
    solution = coding_agent.solve_question(formatted_question)
    print(f"\n The solution generated by the agent is : \n\n {solution}")
    
    return {"solution" : solution }

CREATING A HUMAN IN THE GRAPH NODE

In [9]:
# This node is ment to add human intput in the graph 

def human_review(state: State):
    """ this is to take teh human feed back on the question generated """


    print("*********** Human review node is running ****************")

    #1. print the current question 
    print(f"The current generated question is : \n\n {state['question']}")
    #2. take the human input
    feedback = input("Please provide your feedback on the question generated : ")
    
    return {"human_feedback": feedback.lower()}

Creating a router node

In [10]:
def router(state: State):
    """ this is a router node to decide the flow of the graph based on the human feedback """
    
    print("*********** Router node is running ****************")

    feedback = state.get("human_feedback", "")
    
    # CHECK IF THE FEEDBACK IS POSITIVE OR NOT 
    if feedback == "yes":
        return "solver"
    elif feedback == "no":
        return "question_crafter"

CREATING THE GRAPH 

In [11]:
# using the stateGraph of lang graph to create a 3 node graph to connect all the agents
from langgraph.graph import START , END , StateGraph

#1. initialize an empty graph with the state object
graph = StateGraph(State) # State is the shared data between all the nodes

#2. add nodes to the graph
graph.add_node("Researcher" , research_node)
graph.add_node("Question Generator" , question_crafter_node)

# adding one more node in the graph to take human feed back
graph.add_node("Human Review" , human_review) 
graph.add_node("Question Solver" , question_solver)

#3. add the edges 
graph.add_edge(START , "Researcher")
graph.add_edge("Researcher" , "Question Generator")
graph.add_edge("Question Generator" , "Human Review") 
#graph.add_edge("Question Generator" , "Question Solver")
graph.add_edge("Question Solver" , END)

# adding conditional edges
graph.add_conditional_edges(
    "Human Review",
    router,
    {
        "solver": "Question Solver", # this is for yes 
        "question_crafter": "Question Generator" # this is for no case
    }
)

#graph.add_edge("Researcher" , END)

complete_dsa_agent = graph.compile()

graph design 2

In [12]:
# 1. Instantiate the graph
graph = StateGraph(State)

# 2. Add all the nodes
graph.add_node("Researcher", research_node)
graph.add_node("Question Generator", question_crafter_node)
graph.add_node("Human Review", human_review)
graph.add_node("Question Solver", question_solver)

# 3. Define the edges
graph.set_entry_point("Researcher")
graph.add_edge("Researcher", "Question Generator")
graph.add_edge("Question Generator", "Human Review")

# The conditional edge now only chooses between solving or regenerating
graph.add_conditional_edges(
    "Human Review",
    router,
    {
        "solver": "Question Solver",
        "question_crafter": "Question Generator"
    }
)

# The ONLY path to the end is from the Question Solver
graph.add_edge("Question Solver", END)

# 4. Compile the graph
complete_dsa_agent = graph.compile()

In [13]:
# printing the graph 

print(complete_dsa_agent.get_graph().print_ascii())

    +-----------+      
    | __start__ |      
    +-----------+      
           *           
           *           
           *           
    +------------+     
    | Researcher |     
    +------------+     
           *           
           *           
           *           
+--------------------+ 
| Question Generator | 
+--------------------+ 
           *           
           *           
           *           
   +--------------+    
   | Human Review |    
   +--------------+    
           .           
           .           
           .           
  +-----------------+  
  | Question Solver |  
  +-----------------+  
           *           
           *           
           *           
      +---------+      
      | __end__ |      
      +---------+      
None


TESTING THE GRAPH WORK FLOW

In [14]:
# --- FINAL TEST BLOCK ---
if __name__ == "__main__":
    import json

    # Define the initial state to kick off the graph
    initial_state = {
        "topic": "key concepts on binary search algorithm and most common questions on it"
    }

    print("🚀 Invoking the LangGraph Application...")
    
    # The .invoke() method runs the graph from the entry point to the end.
    final_state = complete_dsa_agent.invoke(initial_state)

    print("\n--- ✅ Graph Execution Complete ---")
    
    # The final state contains all the accumulated data.
    # We can now access the generated question and its solution.
    
    print("\n\n--- Generated Question ---")
    print(json.dumps(final_state['question'], indent=2))
    
    print("\n\n--- Generated Solution ---")
    print(json.dumps(final_state['solution'], indent=2))

🚀 Invoking the LangGraph Application...
********* Researcher node is running**********
Output of gemini api is : 
['The Medium article "Top 10 Most Asked Binary Search Interview Questions" by Kirti Arora discusses ten frequently encountered binary search problems in technical interviews, emphasizing the algorithm\'s O(log n) time complexity for sorted data structures.\n\nThe article covers:\n1.  **Basic Binary Search**: Implementation for finding a target element in a sorted array, typically using iterative or recursive methods. A Java iterative solution is provided.\n2.  **Find the First and Last Position of an Element in a Sorted Array**: Requires modifying binary search to locate both the first and last occurrences of a target value, returning `[-1, -1]` if not found. Separate helper functions `findFirst` and `findSecond` are used, adjusting the search range after finding a potential match.\n3.  **Search Insert Position**: Determines where a target value should be inserted into a so

In [15]:
print(final_state['research_json'])

{
  "title": "Comprehensive Guide to Binary Search",
  "description": "Binary search is a highly efficient algorithm for finding a target element within a sorted array or list. It employs a 'divide and conquer' strategy, repeatedly halving the search space until the target is found or the search space is exhausted, significantly reducing time complexity.",
  "algorithm_steps": [
    {},
    {},
    {},
    {},
    {},
    {}
  ],
  "coding_implementations": [
    {},
    {}
  ],
  "specialized_implementations": [
    {},
    {},
    {},
    {},
    {},
    {},
    {},
    {},
    {},
    {}
  ],
  "benefits": [
    {},
    {},
    {},
    {},
    {}
  ]
}
